In [ ]:
import csv
import bcrypt
import re
from getpass import getpass
import requests
import random

# File to store user data
USER_FILE = 'regno.csv'

# Password validation 
def validate_password(password):
    if len(password) < 8:
        return False
    if not any(char.isupper() for char in password):
        return False
    if not any(char.islower() for char in password):
        return False
    if not any(char.isdigit() for char in password):
        return False
    if not any(char in "!@#$%^&*()-_+=" for char in password):
        return False
    return True


# Hash a password using bcrypt
def hash_password(password):
    return bcrypt.hashpw(password.encode('utf-8'), bcrypt.gensalt())



# Check if the user exists in the CSV file
def check_user_exists(email):
    try:
        with open(USER_FILE, 'r') as file:
            reader = csv.reader(file)
            for row in reader:
                if row[0] == email:
                    return row
    except FileNotFoundError:
        return None
    return None



#Email Validation
def validate_email(email):
    """
    Validates an email address based on the common format:
    - Must contain '@' and '.' after '@'
    - Cannot start with special characters
    - Cannot have consecutive special characters like '..'
    """
    email_regex = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    
    if re.match(email_regex, email):
        return True
    else:
        return False



# Sign-up: Register a new user
def sign_up():
    print("........SIGN_UP.......")
    print()
    #check email validation
    while True:
        email = input("Enter your email: ")

        if validate_email(email):
            break
        else:
            print("Invalid email format. Please enter a valid email.")
            
    # Check if user already exists
    if check_user_exists(email):
        print("User already exists. Try logging in.")
        return
    
    password = getpass("Enter password: ")
    
    if not validate_password(password):
        print("Password must be at least 8 characters, contain an uppercase letter, a lowercase letter, a digit, and a special character.")
        return
    
    hashed_password = hash_password(password)
    security_question = input("Enter a security question: ")
    security_answer = input("Enter the answer to the security question: ")
    
    # Write the new user data to the CSV file
    with open(USER_FILE, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([email, hashed_password.decode('utf-8'), security_question, security_answer])
    print("Registration successful! You can now log in.")



def generate_otp():
    return random.randint(1000, 9999)  # 4-digit OTP



# Sign-in: Login function with attempt limitation
def sign_in():
    print("........SIGN_IN.......")
    print()
    attempts = 5
    
    while attempts > 0:
        email = input("Enter email: ")
        password = getpass("Enter password: ")
        
        user = check_user_exists(email)
        if user:
            hashed_password = user[1].encode('utf-8')
            if bcrypt.checkpw(password.encode('utf-8'), hashed_password):
                otp = generate_otp()
                print(f"Captcha (OTP): {otp}")
                
                # Ask user to enter the captcha
                entered_otp = input("Enter the captcha (OTP): ")
                
                # Validate the OTP
                if str(otp) == entered_otp:
                    print("Login successful!")
                    return True
                else:
                    print("Incorrect captcha. Please try again.")
             
            else:
                attempts -= 1
                print(f"Invalid password. {attempts} attempts remaining.")
        else:
            attempts -= 1
            print(f"Email not found. {attempts} attempts remaining.")
        
        if attempts == 0:
            if input("Would you like to reset your password? (yes/no) ").strip().lower() == "yes":
                forgot_password(email)
            else:
               print("Too many failed attempts. Exiting.")
            break
    return False


    
# Forgot Password: Allow password reset if security question is answered correctly
def forgot_password(email):
    
    print("........PASSWORD_RECOVERY.......")
    print()
    
    user = check_user_exists(email)
    if not user:
        print("No user found with this email.")
        return
    
    security_question = user[2]
    security_answer = input(f"Answer the security question: {security_question}: ")
    
    if security_answer == user[3]:
        print("Security answer verified. You can reset your password.")
        
        new_password = getpass("Enter new password: ")
        if not validate_password(new_password):
            print("Password must meet the security requirements.")
            return
        
        # Update CSV with new hashed password
        hashed_new_password = hash_password(new_password)
        
        with open(USER_FILE, 'r') as file:
            users = list(csv.reader(file))
        
        with open(USER_FILE, 'w', newline='') as file:
            writer = csv.writer(file)
            for row in users:
                if row[0] == email:
                    row[1] = hashed_new_password.decode('utf-8')  # Update the password
                writer.writerow(row)
        
        print("Password reset successful! You can now log in with your new password.")
    else:
        print("Incorrect security answer. Please try again.")




# Function to get weather data from OpenWeatherMap
def get_weather(city_name, api_key):
    base_url = "http://api.weatherapi.com/v1/current.json?"
    complete_url = base_url + "key=" + api_key + "&q=" + city_name
    
    try:
        response = requests.get(complete_url)
        # print(f"Response status code: {response.status_code}")  # To check the response code
        response.raise_for_status()  # Raises HTTPError for bad responses
        
        # If status code is 200, process the response
        data = response.json()
        # print(data)  # To inspect the JSON response
        
        if 'location' in data and 'current' in data:
            location = data['location']['name']
            temperature = data['current']['temp_c']  # Temperature in Celsius
            weather_desc = data['current']['condition']['text']
            humidity = data['current']['humidity']
            wind_speed = data['current']['wind_kph']  # Wind speed in km/h
            
            # Display the weather information
            print(f"Weather in {location}:")
            print()
            print(f"Temperature: {temperature}°C")
            print(f"Weather description: {weather_desc}")
            print(f"Humidity: {humidity}%")
            print(f"Wind Speed: {wind_speed} km/h")
        else:
            print("City not found or no weather data available.")
    
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"Request error occurred: {req_err}")





# Main menu to select action
def main_menu():
    while True:
        print(" --Welcome to the Weather Forecast Application-- ")
        print()
        print("1. Sign Up (New User)")
        print("2. Sign In (Existing User)")
        print("3. Forgot Password")
        print("4. logout ")
        
        choice = input("Enter your choice (1, 2, 3, or type 'logout'): ").strip().lower()
        print()
        
        if choice == '1':
            sign_up()
        elif choice == '2':
            if sign_in():
                # After successful login, show another menu with options like weather forecast or logout
                user_session()
        elif choice == '3':
            email=input(" enter your email: ")
            forgot_password(email)
        elif choice == 'logout':
            print("Goodbye!")
            break
        else:
            print("Invalid choice. Please choose a valid option.")



# After successful login, user session with 'logout' option
def user_session():
    while True:
        print("\nYou are logged in.")
        print("1. Check Weather Forecast")
        print("2. Logout")
        
        choice = input("Enter your choice (1 for Weather, 2 for Logout): ").strip().lower()
        
        if choice == '1':
            api_key = "7fe51b3c67214467ab974325240610"  # Replace with your OpenWeatherMap API key
            city_name = input("Enter city name to get weather: ")
            print()
            get_weather(city_name, api_key)
  # Placeholder for the weather API integration
        elif choice == '2' or choice == 'logout':
            print("Logging out. Returning to the main menu.")
            break
        else:
            print("Invalid choice. Please select a valid option.")

# Start the application
main_menu()


 --Welcome to the Weather Forecast Application-- 

1. Sign Up (New User)
2. Sign In (Existing User)
3. Forgot Password
4. logout 


Enter your choice (1, 2, 3, or type 'logout'):  2



........SIGN_IN.......



Enter email:  asdf
Enter password:  ········


Email not found. 4 attempts remaining.


...

Email not found. 1 attempts remaining.


Enter email:  Sreeeegmail.com
Enter password:  ········


Email not found. 0 attempts remaining.


Would you like to reset your password? (yes/no)  no


Too many failed attempts. Exiting.
 --Welcome to the Weather Forecast Application-- 

1. Sign Up (New User)
2. Sign In (Existing User)
3. Forgot Password
4. logout 
